# **Geometric Brownian Motion**
---
- stochastic process to help better define and expound upon Black-Scholes Model 

### Import *.py* Libraries

In [1]:
# only shows data first time - restart kernel 
import CONFIG as c
import BLACK_SCHOLES as bs

Stock: SPY 
 Expiration: 12-18-2022 
 Strike Price: 370 
 Data Source: yahoo 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2021-04-29 to 2022-04-28
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   High              253 non-null    float64
 1   Low               253 non-null    float64
 2   Open              253 non-null    float64
 3   Close             253 non-null    float64
 4   Volume            253 non-null    float64
 5   Adj Close         253 non-null    float64
 6   close_day_before  252 non-null    float64
 7   returns           252 non-null    float64
dtypes: float64(8)
memory usage: 17.8 KB
None
 
Variables: 
 s = Last Close Price: 427.80999755859375 
 k = Strike Price: 370 
 t = Time to Maturity: 0.6383561643835617 
 r = Risk-Free-Rate: 0.0286299991607666 
 Sigma: 0.15948160363892538
Risk-Adjusted Probability (d1): 1.3464809910807405 (d2): 1.2190596644800977
Individual Fu

### Import Libraries

In [18]:
import pandas as pd
import numpy as np 

from datetime import datetime,date
from scipy.stats import norm
from math import log, sqrt, pi, exp

---
---
## Stochastic Differential Equation: 
---
---
#### Function: `dSt = r*Stdt + sigma*StdZt`
##### Variables: 
> - St: index level at date *t*
> - r: risk-free rate
> - sigma: volatility 
> - Z: standard normally distributed random variable 

In [19]:
# Class to Model Market Environment Relevant for Valuation 
class MarketEnvironment(object): 
    def __init__(self, name, pricing_date): 
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self,key,constant): 
        self.constants[key] = constant 
    def get_constant(self,key): 
        return self.constants[key]
    
    def add_list(self, key, list_object): 
        self.lists[key] = list_object
    def get_list(self, key):
        return self.lists[key]
    
    def add_curve(self, key, curve): 
        self.curves[key] = curve
    def get_curve(self, key): 
        return self.curves[key]
    
    def add_environment(self, env): 
        for key in env.constants: 
            self.constants[key] = env.constants[key]
        for key in env.lists: 
            self.lists[key] = env.lists[key]
        for key in env.curves: 
            self.curves[key] = env.curves[key]

In [28]:
# Simulation Class to Simulate Various Models
class SimulationClass(object): 
    def __init__(self, name, mar_env, corr): 
        try: 
            self.name = name
            self.pricing_date = mar_env.pricing_date
            self.initial_value = mar_env.get_constant('initial_value')
            self.volatility = mar_env.get_constant('volatility')
            self.final_date = mar_env.get_constant('final_date')
            self.currency = mar_env.get_constant('currency')
            self.frequency = mar_env.get_constant('frequency')
            self.paths = mar_env.get_constant('paths')
            self.discount_curve = mar_env.get_constant('discount_curve')
        
            try: 
                self.time_grid = mar_env.get_list('time_grid')
            except: 
                self.time_grid = None
        
            try: 
                self.special_dates = mar_env.get_list('special_dates')
            except: 
                self.special_dates = []
        
            self.instrument_values = None
            self.correlated = corr
            if corr is True: 
                self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
                self.rn_set = mar_env.get_list('rn_set')[self.name]
                self.random_numbers = mar_env.get_list('random_numbers')
        except: 
            print("Error Parsing Market Environment")
            
            
        
    def generate_time_grid(self): 
        start = self.pricing_date
        end = self.final_date
        
        time_grid = pd.date_range(start=start, end=end, freq=self.frequency)
        time_grid = list(time_grid)
        
        if start not in time_grid: 
            time_grid.insert(0,start)
        if end not in time_grid:
            time_grid.append(end)
        if len(self.special_dates) > 0: 
            time_grid.extend(self.special_dates)
            time_grid = list(set(time_grid))
            time_grid.sort()
        self.time_grid = np.array(time_grid) 
        
        
    def get_instrument_values(self, fixed_seed=True): 
        if self.instrument_values is None: 
            # self.generate_paths in GeometricBrownianMotion
            self.generate_paths(fixed_seed=fixed_seed, day_count=365)
        elif fixed_seed is False: 
            self.generate_paths(fixed_seed=fixed_seed, day_count=365)
        return self.instrument_values

In [32]:
# Generates Simulated Paths
# Super Method -> calls SimulationClass
class GeometricBrownianMotion(SimulationClass): 
    
    def __init__(self, name, mar_env, corr=False): 
        super(GeometricBrownianMotion, self).__init__(name, mar_env, corr)
        
    
    def update(self, initial_value=None, volatility=None, final_date=None): 
        if initial_value is not None: 
            self.initial_value = initial_value
        if volatility is not None: 
            self.volatility = volatility
        if final_date is not None: 
            self.final_date = final_date
    
    def generate_paths(self, fixed_seed=False, day_count=365):
        if self.time_grid is None: 
            self.generate_time_grid()
        
        M = len(self.time_grid)
        J = self.paths
        paths = np.zeros((M,J))
        
        if not self.correlated: 
            rand = sn_random_numbers((1,M,J), fixed_seed=fixed_seed)
        else: 
            rand = self.random_numbers
        
        short_rate = self.discount_curve.short_rate
        for t in range(1, len(self.time_grid)):
            if not self.correlated:
                ran = rand[t]
            else: 
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
            dt = (self.time_grid[t] - self.time_grid[t-1]).days/day_count
            paths[t] = paths[t-1] * np.exp((short_rate - 0.5 * self.volatility**2) * dt + self.volatility*np.sqrt(dt)*ran)
        self.instrument_values = paths 

In [33]:
class ConstantShortRate(object):
    
    def __init__(self, name, short_rate): 
        self.name=name
        self.short_rate = short_rate 
        if short_rate < 0: 
            raise ValueError("Negative Short Rate")
            
    def get_year_deltas(date_list, day_count=365): 
        start = min(date_list)
        delta_list = [(date - start).days / day_count for date in date_list]
        return np.array(delta_list)
    
    def get_discount_factors(self, date_list, dtobjects=True): 
        if dtobjects is true: 
            dlist = get_year_deltas(date_list)
        else: 
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * -dlist)
        return np.array((date_list, dflist)).T

In [34]:
me_gbm = MarketEnvironment('me_gbm', datetime(2020,1,1))

me_gbm.add_constant('initial_value',36)
me_gbm.add_constant('volatility', 0.1)
me_gbm.add_constant('final_date', datetime(2020,12,31))
me_gbm.add_constant('currency', 'EUR')
me_gbm.add_constant('frequency', 'M')
me_gbm.add_constant('paths', 10000)

csr = ConstantShortRate('csr', 0.05)
me_gbm.add_curve('discount_curve', csr)

gbm = GeometricBrownianMotion('gbm', me_gbm)

gbm.generate_time_grid()
paths_1 = gbm.get_instrument_values()
gbm.update(volatility=0.5)
paths_2 = gbm.get_instrument_values(fixed_seed=False)

Error Parsing Market Environment


AttributeError: 'GeometricBrownianMotion' object has no attribute 'special_dates'